In [1]:
import pandas as pd

path = r'C:\Users\juan.sossa\projects_integrait\DNDI\data\db3bea42-add2-4185-9a08-d8a8a2622277.xlsx'
df_root = pd.read_excel(path)

In [2]:
df= df_root[df_root['name']== 'Medical Or Surgical History (other than Leishmaniasis)']
lista_sujetos = df['Participante'].unique()
df = df[['name', 'Visit', 'activityState', 'Participante', 'Estado del Participante', 'Campo', 'Valor']]

In [5]:
df_lead_egc_undefined = df_root[df_root['name']=='Medical Or Surgical History (other than Leishmaniasis)']
df_lead_egc_undefined = df_lead_egc_undefined[['Visit','Participante', 'Campo', 'Valor']]
df_lead_egc_undefined = df_lead_egc_undefined[df_lead_egc_undefined['Campo']=='Onset Date/First Diagnosis/Surgery']
df_lead_egc_undefined = df_lead_egc_undefined[['Visit','Participante','Valor']]
df_lead_egc_undefined = df_lead_egc_undefined.rename(columns={'Participante':'Subject', 'Valor':'abnormal_specify'})

In [6]:
df_lead_egc_undefined

,Visit,Subject,abnormal_specify
1896,Screening Visit,011002,17-FEB-2007
1897,Screening Visit,011002,7-MAY-2019
1898,Screening Visit,011002,20-MAY-2004
1899,Screening Visit,011002,7-APR-2022
1900,Screening Visit,011005,UNK-JAN-2022
1901,Screening Visit,011005,8-APR-2016
1902,Screening Visit,011005,16-FEB-2008
1903,Screening Visit,012011,UNK-MAR-2022
1904,Screening Visit,012017,1-JAN-2008
1905,Screening Visit,012017,2-JAN-2008


In [3]:
prueba = df[['Campo', 'Valor']].T
columns_analize = prueba.iloc[0].unique()
columns_analize

array(['Are there any relevant medical history or surgical history ?',
       'Medical/Surgical History/Current Condition',
       'Onset Date/First Diagnosis/Surgery', 'Is Condition Ongoing?',
       'End Date', 'Severity', 'Frequency', 'Currently treated?'],
      dtype=object)

In [ ]:
from datetime import datetime
from revision_fechas import revision_fecha
import warnings

warnings.filterwarnings('ignore')

lista_revision = []

# fecha_inicio = datetime.strptime('19-06-2023', "%d-%m-%Y")
# fecha_fin =  datetime.strptime('31-10-2023', "%d-%m-%Y")

for sujeto in lista_sujetos:
    sujeto_principal = df[df['Participante']==sujeto]

    for visita in sujeto_principal.Visit.unique():
        pru_1 = sujeto_principal[sujeto_principal['Visit']==visita]
        pru = pru_1
        pru = pru[['Campo', 'Valor']].T
        new_columns = pru.iloc[0]
        pru = pru[1:].set_axis(new_columns, axis=1)
        pru['Subject'] = sujeto
        pru['Visit'] = visita
        pru['status'] = pru_1['activityState'].unique()

        for index, row in pru.iterrows():
            status = row['status']
            if status == 'INCOMPLETE' or status == 'PENDING_QUERIES':
                pass
            else:
                any_relevant_medical = row['Are there any relevant medical history or surgical history ?']
                medical_surgical = row['Medical/Surgical History/Current Condition']
                onset_date = row['Onset Date/First Diagnosis/Surgery']
                condition_ongoing = row['Is Condition Ongoing?']
                end_date = row['End Date']
                severity = row['Severity']
                frequency = row['Frequency']
                currently_treated = row['Currently treated?']